In [ ]:
import os
import re
import sys
import codecs
import hashlib
from scapy.all import *

In [ ]:
password = # RTSP 접속 비밀번호를 기입합니다. # Brute-Forcing 공격을 수행하면, 비밀번호를 찾는 코드로 변환 가능

# .pcapng 파일의 경로를 설정합니다.
pcap_file_dir = # 경로를 삽입합니다.

# .pcapng 파일의 이름을 찾습니다.
pcap_file_names = os.listdir(pcap_file_dir)

In [ ]:
# pcap 파일을 읽어들입니다.
pcap_file_data = ""
with codecs.open(os.path.join(pcap_file_dir,pcap_file_names[0]), 'r', encoding='utf-8', errors='ignore') as f:
    pcap_file_data = f.read()

In [ ]:
# 찾아야 하는 바이트 열에 대한 패턴을 입력합니다.
method_pattern = "(SETUP|DESCRIBE)"
username_pattern = "(username=\"([a-zA-Z0-9]+))"
realm_pattern = "(realm=\"([a-zA-Z0-9]+))"
nonce_pattern = "(nonce=\"([a-zA-Z0-9]+))"
uri_pattern = "(uri=\"(.+)\"\,)"
response_pattern = "(response=\"([a-zA-Z0-9]+))"

print("-"*90)
print("<<Digest 방식 재현을 위한 데이터 준비 완료.>>")
# 바이트열에서 찾아야 하는 값을 찾습니다.
method = re.search(method_pattern, pcap_file_data)
if method:
    method = method.group(1)
else:
    print("RTSP 연결 과정이 설정되지 않았습니다.")
    sys.exit(1)
print("method: ",method)

username = re.search(username_pattern, pcap_file_data)
if username:
    username = username.group(2)
print("username: ",username)

realm = re.search(realm_pattern, pcap_file_data)
if realm:
    realm = realm.group(2)
print("realm: ",realm)

nonce = re.search(nonce_pattern, pcap_file_data)
if nonce:
    nonce = nonce.group(2)
print("nonce: ",nonce)

uri = re.search(uri_pattern, pcap_file_data)
if uri:
    uri = uri.group(2)
print("uri: ",uri)

response = re.search(response_pattern, pcap_file_data)
if response:
    response = response.group(2)
print("response: ",response)

In [ ]:
# 첫번째 단계: md5(username:realm:password)
input_str = username+":"+realm+":"+password
input_str = input_str.encode('utf-8')

md5_1 = hashlib.md5()
md5_1.update(input_str)
md5_1 = md5_1.hexdigest()

# 두번째 단계: md5(DESCRIBE:rtsp://ip/urn)
input_str = method+":"+uri
input_str = input_str.encode('utf-8')

md5_2 = hashlib.md5()
md5_2.update(input_str)
md5_2 = md5_2.hexdigest()

# 세번째 단계: md5(hash1:nonce:hash2)
input_str = md5_1+":"+nonce+":"+md5_2
input_str = input_str.encode('utf-8')

md5_3 = hashlib.md5()
md5_3.update(input_str)
md5_3 = md5_3.hexdigest()

if md5_3 == response: 
    print("Responce 일치!!")
else:
    print("Responce 불 일치!!")

Responce 일치!!


## 참고자료
- [RTSP 인증 방식](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=bcknightt&logNo=220006031922)
- [Digest 방식](https://en.wikipedia.org/wiki/Digest_access_authentication)

## 제작정보
- 제작자: 김동영 (klgh1256@s.hallym.ac.kr)
- 제작일자: 2022년 08월 22일